Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic Regression with L2 Regularization

In [41]:
def accuracy_score(predictions, truth):
    return 100.*np.sum(np.argmax(predictions, 1) == np.argmax(truth, 1))/predictions.shape[0]

In [90]:
lr_graph    = tf.Graph()
n_classes   = train_labels.shape[1]
batch_size  = 128
lam         = 0.01 # Regularization parameter

with lr_graph.as_default():
    # Constants for the data
    tf_train_data   = tf.placeholder(tf.float32, shape = [batch_size, train_dataset.shape[1]])
    tf_train_labels = tf.placeholder(tf.float32, shape = [batch_size, n_classes])
    tf_valid_data   = tf.constant(valid_dataset)
    tf_test_data    = tf.constant(test_dataset)
    
    # Variables for weights and biases
    weights         = tf.Variable(tf.truncated_normal([train_dataset.shape[1], n_classes]))
    biases          = tf.Variable(tf.zeros([n_classes]))
    
    # Inference and loss calculation
    logits = tf.matmul(tf_train_data, weights) + biases
    loss   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits)) + lam*tf.nn.l2_loss(weights)
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Feedback information
    tf_train_predictions = tf.nn.softmax(logits)
    tf_valid_predictions = tf.nn.softmax(tf.matmul(tf_valid_data, weights) + biases)
    tf_test_predictions  = tf.nn.softmax(tf.matmul(tf_test_data, weights) + biases)

In [91]:
n_steps = 5000

with tf.Session(graph = lr_graph) as session:
    # Initialize variables
    tf.global_variables_initializer().run()
    print('Session initialized. Running...')
    for step in range(n_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size) # To vary the minibatch taken
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_data : batch_data, tf_train_labels : batch_labels} 
        _, l, train_predictions = session.run([optimizer, loss, tf_train_predictions], feed_dict = feed_dict)
        if step % 500 == 0:
            print('Step {}'.format(step))
            print('Loss: {}'.format(l))
            print('Training accuracy: {}%'.format(accuracy_score(train_predictions, batch_labels)))
            print('Validation accuracy: {}%'.format(accuracy_score(tf_valid_predictions.eval(), valid_labels)))
            print('')
            
    print('Testing accuracy: {}%'.format(accuracy_score(tf_test_predictions.eval(), test_labels)))

Session initialized. Running...
Step 0
Loss: 48.784767150878906
Training accuracy: 7.03125%
Validation accuracy: 10.85%

Step 500
Loss: 0.8551467061042786
Training accuracy: 84.375%
Validation accuracy: 81.55%

Step 1000
Loss: 0.6211180090904236
Training accuracy: 85.15625%
Validation accuracy: 81.62%

Step 1500
Loss: 0.996584951877594
Training accuracy: 78.125%
Validation accuracy: 81.97%

Step 2000
Loss: 0.7599500417709351
Training accuracy: 82.03125%
Validation accuracy: 82.1%

Step 2500
Loss: 0.74886155128479
Training accuracy: 81.25%
Validation accuracy: 81.92%

Step 3000
Loss: 0.6802355051040649
Training accuracy: 83.59375%
Validation accuracy: 81.56%

Step 3500
Loss: 0.8542815446853638
Training accuracy: 81.25%
Validation accuracy: 81.27%

Step 4000
Loss: 0.8930375576019287
Training accuracy: 80.46875%
Validation accuracy: 81.44%

Step 4500
Loss: 0.6157508492469788
Training accuracy: 85.9375%
Validation accuracy: 81.02%

Testing accuracy: 84.21%


## Multi-layer Perceptron with L2 Regularization

In [101]:
nn_graph       = tf.Graph()
n_classes   = train_labels.shape[1]
batch_size  = 128
lam_nn      = 0.01 # Regularization parameter
n_relus     = 1024
n_features  = train_dataset.shape[1]

with nn_graph.as_default():
    # Inputs
    tf_train_data   = tf.placeholder(tf.float32, shape = [batch_size, n_features])
    tf_train_labels = tf.placeholder(tf.float32, shape = [batch_size, n_classes])
    tf_valid_data   = tf.constant(valid_dataset)
    tf_test_data    = tf.constant(test_dataset)
    
    # First layer
    weights_1    = tf.Variable(tf.truncated_normal([n_features, n_relus]))
    biases_1     = tf.Variable(tf.zeros([n_relus]))
    activation_1 = tf.nn.relu(tf.matmul(tf_train_data, weights_1) + biases_1)
    
    # Second layer
    weights_2    = tf.Variable(tf.truncated_normal([n_relus, n_classes]))
    biases_2     = tf.Variable(tf.zeros(n_classes))
    activation_2 = tf.matmul(activation_1, weights_2) + biases_2
    
    # Loss
    loss         = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = activation_2)) + lam_nn*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
    
    # Optimizer
    optimizer    = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Feedback
    tf_train_predictions = tf.nn.softmax(activation_2)
    valid_activation_1   = tf.nn.relu(tf.matmul(tf_valid_data, weights_1) + biases_1)
    tf_valid_predictions    = tf.nn.softmax(tf.matmul(valid_activation_1, weights_2) + biases_2)
    
    test_activation_1    = tf.nn.relu(tf.matmul(tf_test_data, weights_1,) + biases_1)
    tf_test_predictions     = tf.nn.softmax(tf.matmul(test_activation_1, weights_2) + biases_2)
    

In [102]:
n_steps = 5000

with tf.Session(graph = nn_graph) as session:
    
    tf.global_variables_initializer().run()
    print('Session initialized. Running...')
    
    for step in range(n_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size) # To vary the minibatch taken
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_data : batch_data, tf_train_labels : batch_labels} 
        _, l, train_predictions = session.run([optimizer, loss, tf_train_predictions], feed_dict = feed_dict)
        
        if step % 500 == 0:
            print('Step {}'.format(step))
            print('Loss: {}'.format(l))
            print('Training accuracy: {}%'.format(accuracy_score(train_predictions, batch_labels)))
            print('Validation accuracy: {}%'.format(accuracy_score(tf_valid_predictions.eval(), valid_labels)))
            print('')
            
    print('Testing accuracy: {}%'.format(accuracy_score(tf_test_predictions.eval(), test_labels)))

Session initialized. Running...
Step 0
Loss: 3478.406982421875
Training accuracy: 7.8125%
Validation accuracy: 29.91%

Step 500
Loss: 21.288034439086914
Training accuracy: 84.375%
Validation accuracy: 84.26%

Step 1000
Loss: 0.723535418510437
Training accuracy: 85.9375%
Validation accuracy: 83.95%

Step 1500
Loss: 0.8553493022918701
Training accuracy: 80.46875%
Validation accuracy: 83.94%

Step 2000
Loss: 0.7460545301437378
Training accuracy: 82.8125%
Validation accuracy: 83.72%

Step 2500
Loss: 0.6778848767280579
Training accuracy: 84.375%
Validation accuracy: 83.85%

Step 3000
Loss: 0.6802613735198975
Training accuracy: 85.9375%
Validation accuracy: 83.81%



KeyboardInterrupt: 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [104]:
n_steps         = 1001
n_train_samples = batch_size*2
rnd_ix          = np.random.choice(range(train_dataset.shape[0]), n_train_samples)
restricted_train_dataset = train_dataset[rnd_ix]
restricted_train_labels  = train_labels[rnd_ix]

with tf.Session(graph = nn_graph) as session:
    # Initialize variables
    tf.global_variables_initializer().run()
    print('Session initialized. Running...')
    for step in range(n_steps):
        offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size) # To vary the minibatch taken
        batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
        batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_data : batch_data, tf_train_labels : batch_labels} 
        _, l, train_predictions = session.run([optimizer, loss, tf_train_predictions], feed_dict = feed_dict)
        if step % 500 == 0:
            print('Step {}'.format(step))
            print('Loss: {}'.format(l))
            print('Training accuracy: {}%'.format(accuracy_score(train_predictions, batch_labels)))
            print('Validation accuracy: {}%'.format(accuracy_score(tf_valid_predictions.eval(), valid_labels)))
            print('')
            
    print('Testing accuracy: {}%'.format(accuracy_score(tf_test_predictions.eval(), test_labels)))

Session initialized. Running...
Step 0
Loss: 3453.11572265625
Training accuracy: 9.375%
Validation accuracy: 22.3%

Step 500
Loss: 20.99122428894043
Training accuracy: 100.0%
Validation accuracy: 64.17%

Step 1000
Loss: 0.340889036655426
Training accuracy: 100.0%
Validation accuracy: 69.0%

Testing accuracy: 75.19%


In [105]:
n_steps         = 1001

with tf.Session(graph = nn_graph) as session:
    # Initialize variables
    tf.global_variables_initializer().run()
    print('Session initialized. Running...')
    for step in range(n_steps):
        offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size) # To vary the minibatch taken
        batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
        batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_data : batch_data, tf_train_labels : batch_labels} 
        _, l, train_predictions = session.run([optimizer, loss, tf_train_predictions], feed_dict = feed_dict)
        if step % 500 == 0:
            print('Step {}'.format(step))
            print('Loss: {}'.format(l))
            print('Training accuracy: {}%'.format(accuracy_score(train_predictions, batch_labels)))
            print('Validation accuracy: {}%'.format(accuracy_score(tf_valid_predictions.eval(), valid_labels)))
            print('')
            
    print('Testing accuracy: {}%'.format(accuracy_score(tf_test_predictions.eval(), test_labels)))

Session initialized. Running...
Step 0
Loss: 3467.64599609375
Training accuracy: 10.15625%
Validation accuracy: 30.32%

Step 500
Loss: 21.01329231262207
Training accuracy: 100.0%
Validation accuracy: 64.29%

Step 1000
Loss: 0.3415870666503906
Training accuracy: 100.0%
Validation accuracy: 68.62%

Testing accuracy: 74.78%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [106]:
nn_do_graph       = tf.Graph()
n_classes   = train_labels.shape[1]
batch_size  = 128
lam_nn      = 0.01 # Regularization parameter
n_relus     = 1024
n_features  = train_dataset.shape[1]

with nn_do_graph.as_default():
    # Inputs
    tf_train_data   = tf.placeholder(tf.float32, shape = [batch_size, n_features])
    tf_train_labels = tf.placeholder(tf.float32, shape = [batch_size, n_classes])
    tf_valid_data   = tf.constant(valid_dataset)
    tf_test_data    = tf.constant(test_dataset)
    
    # First layer
    weights_1    = tf.Variable(tf.truncated_normal([n_features, n_relus]))
    biases_1     = tf.Variable(tf.zeros([n_relus]))
    activation_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_data, weights_1) + biases_1), 0.5)
    
    # Second layer
    weights_2    = tf.Variable(tf.truncated_normal([n_relus, n_classes]))
    biases_2     = tf.Variable(tf.zeros(n_classes))
    activation_2 = tf.matmul(activation_1, weights_2) + biases_2
    
    # Loss
    loss         = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = activation_2)) +
                        lam_nn*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)))
    
    # Optimizer
    optimizer    = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Feedback
    tf_train_predictions = tf.nn.softmax(activation_2)
    valid_activation_1   = tf.nn.relu(tf.matmul(tf_valid_data, weights_1) + biases_1)
    tf_valid_predictions = tf.nn.softmax(tf.matmul(valid_activation_1, weights_2) + biases_2)
    
    test_activation_1    = tf.nn.relu(tf.matmul(tf_test_data, weights_1,) + biases_1)
    tf_test_predictions  = tf.nn.softmax(tf.matmul(test_activation_1, weights_2) + biases_2)
    

In [107]:
n_steps         = 5001

with tf.Session(graph = nn_do_graph) as session:
    # Initialize variables
    tf.global_variables_initializer().run()
    print('Session initialized. Running...'
         )
    for step in range(n_steps):
        
        offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size) # To vary the minibatch taken
        batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
        batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_data : batch_data, tf_train_labels : batch_labels} 
        _, l, train_predictions = session.run([optimizer, loss, tf_train_predictions], feed_dict = feed_dict)
        
        if step % 500 == 0:
            print('Step {}'.format(step))
            print('Loss: {}'.format(l))
            print('Training accuracy: {}%'.format(accuracy_score(train_predictions, batch_labels)))
            print('Validation accuracy: {}%'.format(accuracy_score(tf_valid_predictions.eval(), valid_labels)))
            print('')
            
    print('Testing accuracy: {}%'.format(accuracy_score(tf_test_predictions.eval(), test_labels)))

Session initialized. Running...
Step 0
Loss: 3551.030029296875
Training accuracy: 14.84375%
Validation accuracy: 25.68%

Step 500
Loss: 21.027334213256836
Training accuracy: 100.0%
Validation accuracy: 66.01%

Step 1000
Loss: 0.3474011719226837
Training accuracy: 100.0%
Validation accuracy: 69.04%

Step 1500
Loss: 0.19715064764022827
Training accuracy: 100.0%
Validation accuracy: 68.8%

Step 2000
Loss: 0.18994605541229248
Training accuracy: 100.0%
Validation accuracy: 68.62%

Step 2500
Loss: 0.18563655018806458
Training accuracy: 100.0%
Validation accuracy: 68.91%

Step 3000
Loss: 0.18566232919692993
Training accuracy: 100.0%
Validation accuracy: 68.42%

Step 3500
Loss: 0.18011048436164856
Training accuracy: 100.0%
Validation accuracy: 68.63%

Step 4000
Loss: 0.18307086825370789
Training accuracy: 100.0%
Validation accuracy: 68.51%

Step 4500
Loss: 0.1791398525238037
Training accuracy: 100.0%
Validation accuracy: 68.59%

Step 5000
Loss: 0.18117496371269226
Training accuracy: 100.0%
Val

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [162]:
nn_gd_graph = tf.Graph()
n_classes   = train_labels.shape[1]
batch_size  = 100
lam_nn      = 0.01 # Regularization parameter
n_relus_1   = 1024
n_relus_2   = 256
n_features  = train_dataset.shape[1]

with nn_gd_graph.as_default():
    # Inputs
    tf_train_data   = tf.placeholder(tf.float32, shape = [batch_size, n_features])
    tf_train_labels = tf.placeholder(tf.float32, shape = [batch_size, n_classes])
    tf_valid_data   = tf.constant(valid_dataset)
    tf_test_data    = tf.constant(test_dataset)
    
    # First layer
    weights_1    = tf.Variable(tf.truncated_normal([n_features, n_relus_1]))
    biases_1     = tf.Variable(tf.zeros([n_relus_1]))
    activation_1 = tf.nn.relu(tf.matmul(tf_train_data, weights_1) + biases_1)
    
    # Second layer
    weights_2    = tf.Variable(tf.truncated_normal([n_relus_1, n_relus_2]))
    biases_2     = tf.Variable(tf.zeros([n_relus_2]))
    activation_2 = tf.nn.relu(tf.matmul(activation_1, weights_2) + biases_2)
    
    # Third layer
    weights_3    = tf.Variable(tf.truncated_normal([n_relus_2, n_classes]))
    biases_3     = tf.Variable(tf.zeros([n_classes]))
    activation_3 = tf.matmul(activation_2, weights_3) + biases_3
    
    # Loss
    loss         = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = activation_3)) + lam_nn*(tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_2))
    
    # Optimizer
    optimizer     = tf.train.GradientDescentOptimizer(0.05).minimize(loss) # Increments gstep 

    # Feedback
    tf_train_predictions = tf.nn.softmax(activation_3)
    valid_activation_1   = tf.nn.relu(tf.matmul(tf_valid_data, weights_1) + biases_1)
    valid_activation_2   = tf.nn.relu(tf.matmul(valid_activation_1, weights_2) + biases_2)
    tf_valid_predictions = tf.nn.softmax(tf.matmul(valid_activation_2, weights_3) + biases_3)
    
    test_activation_1    = tf.nn.relu(tf.matmul(tf_test_data, weights_1,) + biases_1)
    test_activation_2    = tf.nn.relu(tf.matmul(test_activation_1, weights_2) + biases_2)
    tf_test_predictions  = tf.nn.softmax(tf.matmul(test_activation_2, weights_3) + biases_3)

In [163]:
n_steps = 20000
import sys

with tf.Session(graph = nn_gd_graph) as session:
    
    tf.global_variables_initializer().run()
    print('Session initialized. Running...')
    
    for step in range(n_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size) # To vary the minibatch taken
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_data : batch_data, tf_train_labels : batch_labels} 
        _, l, train_predictions = session.run([optimizer, loss, tf_train_predictions], feed_dict = feed_dict)
        
        if step % 500 == 0:
            sys.stdout.write('\r' + 'Step {}\nLoss: {}\nTraining accuracy: {}%\nValidation accuracy: {}%\n\n'.format(step, l,
                                                                                                                   accuracy_score(train_predictions, batch_labels),
                                                                                                                   accuracy_score(tf_valid_predictions.eval(), valid_labels)))
    
    print('Testing accuracy: {}%'.format(accuracy_score(tf_test_predictions.eval(), test_labels)))

Session initialized. Running...
Step 0
Loss: 4244.349609375
Training accuracy: 13.0%
Validation accuracy: 17.34%

Step 500
Loss: 775.3734130859375
Training accuracy: 36.0%
Validation accuracy: 32.41%

Step 1000
Loss: 470.8066711425781
Training accuracy: 41.0%
Validation accuracy: 45.27%

Step 1500
Loss: 285.82342529296875
Training accuracy: 48.0%
Validation accuracy: 54.02%

Step 2000
Loss: 173.5347442626953
Training accuracy: 64.0%
Validation accuracy: 59.96%

Step 2500
Loss: 105.47322845458984
Training accuracy: 69.0%
Validation accuracy: 68.97%

Step 3000
Loss: 64.32654571533203
Training accuracy: 72.0%
Validation accuracy: 74.85%

Step 3500
Loss: 39.14701843261719
Training accuracy: 78.0%
Validation accuracy: 77.06%

Step 4000
Loss: 24.018474578857422
Training accuracy: 81.0%
Validation accuracy: 79.93%

Step 4500
Loss: 14.828399658203125
Training accuracy: 84.0%
Validation accuracy: 80.88%

Step 5000
Loss: 9.101317405700684
Training accuracy: 84.0%
Validation accuracy: 82.09%

Ste